In [2]:
import os

In [37]:
import pickle

In [5]:
import textract

In [25]:
fd = os.path.abspath(os.path.join('..'))

In [32]:
raw_data_path = os.path.join(fd, "data/raw")
int_data_path = os.path.join(fd, "data/interim")

In [34]:
file_names = [fn for fn in os.listdir(raw_data_path) if fn.endswith(".pdf")]

In [35]:
doc_names = [fn.strip(".pdf") for fn in file_names]
doc_names

['DOC_0005976599',
 'DOC_0005976573',
 'DOC_0005976581',
 'DOC_0005976612',
 'DOC_0005976587']

In [31]:
texts = [textract.process(os.path.join(data_path, fn), method="tesseract", language="eng") for fn in file_names]

In [41]:
for dn,txt in zip(doc_names, texts):
    with open(os.path.join(int_data_path, dn + '.bin'), 'wb') as f1:
        pickle.dump(texts, f1)

In [45]:
doc_names[0]

'DOC_0005976599'

In [46]:
texts[0][:1500]

b'Dec\xe2\x80\x98assmed In Pan - Sanlllled Copy Approved for Release 2016/04/27 : CIA-RDP79TDDBSSADDS7Q00280001-3\n/ K / \\\n94/\n9/\n\n \n\n \n\nT/oe sz\'dmt\xe2\x80\x99y Daily Brief\n\n4 February 1969\n\n18\n\n50x1\n2\n\nDec\xe2\x80\x98assmed In Pal\xe2\x80\x9c \xc2\xbb Sanmxed Copy Approved for Release 2018/04/27 V C|A-RDP79TUUQSSADDS7OOZSOOO1-3\n\nDeclass\xe2\x80\x98lied in Pan - sanitized Copy Approved for Release 2016/04/27 : CIA-RDP79T00936A006700280001-3 _\n\n \n\n:2 /7\n\n@252\xe2\x80\x9d:\n\n \n\n   \n   \n    \n  \n\n \n\nLEBANON\n\nEevul\n\nDxmascus\n\nMediterranean Sea\n\n__, /\namp\n\n/Ymmuk n\n\n   \n  \n \n  \n\n    \n \n   \n   \n      \n \n   \n\nISRAEL r\\\n[/3\nW wssr\n"\xe2\x80\x9c. \xe2\x80\x98 \\amx \xc2\xa7 .n\n_J,_ Am.\xe2\x80\x9d .~ 732-\nEr\nJew am.\xe2\x80\x9c\nand\nmu my; / 5"\n\xe2\x80\x9c3.534 msm\n.2\n\nJORDAN\n\nSINAI\nPENINSULA\n\nqg, ~25-\n\nARABIA\n\n\'Shan-n =an\nShiykh\n\nHugh-a: \\\n\n \n\n \n\n \n\n \n\nmu m\n\nDeclassmed In Part - sanitized Copy

In [48]:
str(texts[0]).lower().find('late notes')

1296

In [50]:
str(texts[0])[1290:1500]

"69\\n\\nLATE NOTES FOR THE PRESIDENT\\'S DAILY BRIEF 0F\\nyFEBRUARY 1969\\n\\nI. MAJOR PROBLEMS\\n\\nMIDDLE EAST\\n\\nThere is nothing of significance to reportl\\n\\nEUROPE\\n\\nThere is nothing of significance to report.\\n"